In [1]:
import wrds
import pandas as pd
import statsmodels.api as sm
import numpy as np
from sas7bdat import SAS7BDAT
import datetime as dt
from pandas.tseries.offsets import *
from statsmodels.regression.rolling import RollingOLS
import pyreadstat
from scipy import stats
import dask.dataframe as dd

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/dask/dataframe/__init__.py:49: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [2]:
def wavg(group, avg_name, weight_name):
    d = group[avg_name]
    w = group[weight_name]
    try:
        return (d * w).sum() / w.sum()
    except ZeroDivisionError:
        return np.nan


def normalize_k(group):

    beta_rank = group['rank']
    beta_avg_rank = group['rank_avg']

    diff_sum = abs(beta_rank - beta_avg_rank).sum()
    k = 2 / diff_sum if diff_sum != 0 else 0

    return k

def vector_vwt(group, vector_name, weight_name):
    scalar = (group[vector_name] * group[weight_name]).sum()
    return scalar


In [3]:
AQR_path = "/Users/benben/Desktop/RA/Task4 Betting against Beta/BAB For US Equity.xlsx"
BAB_AQR = pd.read_excel(AQR_path)
BAB_AQR['date'] = pd.to_datetime(BAB_AQR['date'], format='%Y%m', errors='coerce')
BAB_AQR = BAB_AQR.dropna(subset=['date'])
BAB_AQR['month'] = BAB_AQR['date'].dt.to_period('M')
BAB_AQR['month'] = BAB_AQR['month'].astype(str)

BAB_AQR = BAB_AQR[['month','US_BAB']]

In [4]:
ff_path ="/Users/benben/Desktop/RA/Task4 Betting against Beta/F-F_Research_Data_Factors_Monthly.CSV"
_ff = pd.read_csv(ff_path)
_ff.columns =['month','mkt-rf','smb','hml','rf']
_ff['month'] = pd.to_datetime(_ff['month'], format='%Y%m').dt.strftime('%Y-%m')
_ff['month'] = _ff['month'].astype(str)
_ff[['mkt-rf', 'smb', 'hml','rf']] = _ff[['mkt-rf', 'smb', 'hml','rf']]/100

In [5]:
crsp_path = "/Users/benben/Desktop/RA/Task4 Betting against Beta/crsp daily.sas7bdat"
crsp, meta = pyreadstat.read_sas7bdat(crsp_path)
crsp.columns = crsp.columns.str.lower()
crsp[['permno','permco','shrcd','exchcd',]] = crsp[['permno','permco','shrcd','exchcd']].astype(int)
crsp['date'] = pd.to_datetime(crsp['date']).dt.date
crsp = crsp[(crsp['shrcd']==10)|(crsp['shrcd']==11)]

dfs = crsp.dropna(subset=['ret','prc','shrout'])

"""
Derive market cap
"""
dfs['mkt_cap'] = dfs['shrout'] * abs(dfs['prc'])
dfs['date'] = pd.to_datetime(dfs['date'], errors='coerce')
dfs.dropna(subset=['date'], inplace=True)


/var/folders/vt/x4z4r3fd0xg5nszdcjd244900000gn/T/ipykernel_1748/677262450.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['mkt_cap'] = dfs['shrout'] * abs(dfs['prc'])
/var/folders/vt/x4z4r3fd0xg5nszdcjd244900000gn/T/ipykernel_1748/677262450.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs['date'] = pd.to_datetime(dfs['date'], errors='coerce')
/var/folders/vt/x4z4r3fd0xg5nszdcjd244900000gn/T/ipykernel_1748/677262450.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a s

In [6]:
time_periods = {
    '1926-1959': ('1926-01-01', '1960-01-01'),
    '1960-1993': ('1960-01-01', '1994-01-01'),
    '1994-2024': ('1994-01-01', '2013-12-31')
}

dfs_ny = []
Bab_Port = []

for period, (start_date, end_date) in time_periods.items():
    dfs_period = dfs[(dfs['date'] >= start_date) & (dfs['date'] <= end_date)]

    """
    Estimating ex ante betas
    
    We first need to derive 
     
    1. estimated volatilities for individual stocks and the market use:
                                                        one-year (252d) rolling window, at least 120 trading data
    
    2, correlation between stocks and the market use:
                                        five-year (1260d) rolling window, at least 750 trading data
    """

    dfs_period = dfs_period.sort_values(by=['permno','date'])
    dfs_period['month'] = dfs_period['date'].dt.to_period('M')

    dfs_period['log_ret'] = np.log(dfs_period['ret'] + 1)
    dfs_period['log_ret3d'] = dfs_period.groupby('permno')['log_ret'].rolling(window=3).sum().reset_index(level=0, drop=True)
    dfs_period['stk_month_ret'] = np.exp(dfs_period.groupby(['permno','month'])['log_ret'].transform('sum')) -1

    mkt_ret = dfs_period.groupby('date').apply(wavg, 'ret', 'mkt_cap').reset_index(name='mkt_ret')
    mkt_ret = mkt_ret.sort_values(by=['date'])
    mkt_ret['date'] = pd.to_datetime(mkt_ret['date'], errors='coerce')
    mkt_ret.dropna(subset=['date'], inplace=True)
    mkt_ret['month'] = mkt_ret['date'].dt.to_period('M')
    
    mkt_ret['log_mkt_ret'] = np.log(1 + mkt_ret['mkt_ret'])
    mkt_ret['log_mkt_ret3d'] = mkt_ret['log_mkt_ret'].rolling(window=3).sum().reset_index(level=0, drop=True)
    mkt_ret['mtk_month_ret'] = np.exp(mkt_ret.groupby(['month'])['log_mkt_ret'].transform('sum')) -1
    
    mkt_ret['mkt_volatility'] = mkt_ret['log_mkt_ret'].rolling(window=252, min_periods=120).std()


    dfs_period = dd.merge(dfs_period, mkt_ret[['date', 'mkt_ret','log_mkt_ret3d','mtk_month_ret', 'mkt_volatility']], on='date', )
    
    # dfs = dfs[dfs['exchcd'] == 1|dfs['exchcd'] == 2|dfs['exchcd'] == 3]
    
    
    vol_corr_results = []

    dfs_period = dfs_period.dropna(subset=['log_ret3d', 'log_mkt_ret3d'])
    
    for permno, group in dfs_period.groupby('permno'):
        group = group.set_index('date')
    
        # 要求至少120天数据用于波动率计算，至少750天用于相关性计算
        if len(group) < 750:
            continue
    
        # 1年窗口（252天）计算波动率
        group['volatility'] = group['log_ret'].rolling(window=252, min_periods=120).std()
    
        # 5年窗口（1260天）计算相关性
        group['correlation'] = group['log_ret3d'].rolling(window=1260, min_periods=750).corr(group['log_mkt_ret3d'])
    
        vol_corr_results.append(group[['permno', 'volatility', 'correlation','mkt_volatility']])
    
    # 合并所有股票的波动率和相关性数据
    vol_corr_df = pd.concat(vol_corr_results).dropna().reset_index()
    
    vol_corr_df['pre_ranking_beta'] = vol_corr_df['correlation'] * (vol_corr_df['volatility'] / vol_corr_df['mkt_volatility'])
    
    dfs_period = pd.merge(dfs_period,vol_corr_df[['date','permno','pre_ranking_beta']], on=['date','permno'], how='left')


    """
    Shrink the time series estimate of beta  toward the cross-sectional mean
    Use predetermined shrinkage factor
    """
    
    wi = 0.6
    beta_xs = 1
    
    dfs_period['shrink_beta'] = wi * dfs_period['pre_ranking_beta'] + (1 - wi) * beta_xs
    
    dfs_ny.append(dfs_period[dfs_period['exchcd'] == 1])


    """
    Calculate the median beta of asset classes and months.
    
    Group based on the comparison between:
    
                                 1. Each stock's monthly average beta at the end of the month 
      
                                 2. The monthly median beta according to its exchcd.
                                 
    Rebalance the portfolio monthly.
    """
    
    dfs_period['median_beta'] = dfs_period.groupby('month')['shrink_beta'].transform('median')
    
    dfs_period['avg_beta'] = dfs_period.groupby(['month', 'permno'])['shrink_beta'].transform('mean')
    
    
    dfs_period['beta_group'] = dfs_period.apply(lambda x: 'low' if x['avg_beta'] <= x['median_beta'] else 'high', axis=1)
    dfs_period['beta_group'] = np.where((pd.isna(dfs_period['avg_beta']) | pd.isna(dfs_period['median_beta'])),np.nan ,dfs_period['beta_group'] )
    dfs_period.dropna(subset=['beta_group'], inplace=True)
    
    """
    Derive the securities' monthly beta ascending rank.
    
    Derive ranked weight for each security every month.
    
    """

    #只保留个股每个月的最后一条数据
    dfs_period.sort_values(by=['permno','month','date'], inplace=True)
    dfs_month = dfs_period.drop_duplicates(subset=['permno','month'],keep='last')
    
    dfs_month['lmktcap'] = dfs_month.groupby('permno')['mkt_cap'].shift(1)
    dfs_month['lmktcap'] = np.where(dfs_month['lmktcap'].isna(), dfs_month['mkt_cap']/(1 + dfs_month['stk_month_ret']),dfs_month['lmktcap'])
    
    dfs_month = dfs_month[['month','permno','stk_month_ret','shrink_beta','beta_group','lmktcap','mkt_cap','mtk_month_ret']]
    
    dfs_month.sort_values(by=['month','shrink_beta'], inplace=True)
    dfs_month['rank'] = dfs_month.groupby(['month'])['shrink_beta'].rank(method='first')
    
    dfs_month['rank_avg'] = dfs_month.groupby(['month'])['rank'].transform('mean')
    
    Normalize = dfs_month.groupby(['month']).apply(normalize_k).reset_index(name='normalizing_cons')
    
    dfs_month = pd.merge(dfs_month, Normalize, on=['month'], how='left')
    
    dfs_month['rank_wgt'] = np.where(
        dfs_month['rank'] <= dfs_month['rank_avg'],
        dfs_month['normalizing_cons'] * np.maximum(0, dfs_month['rank_avg'] - dfs_month['rank']),
        dfs_month['normalizing_cons'] * np.maximum(0, dfs_month['rank'] - dfs_month['rank_avg'])
    )

    port_month_ret = dfs_month.groupby(['beta_group','month']).apply(wavg,'stk_month_ret','lmktcap').reset_index(name='port_month_ret')
    
    _ff['month'] = _ff['month'].astype(str)
    
    port_month_ret['month'] = port_month_ret['month'].astype(str)
    
    port_month_ret = pd.merge(port_month_ret, _ff[['month','rf']], on=['month'], how='inner')
    
    port_month_ret['port_excess_ret'] = port_month_ret['port_month_ret']-port_month_ret['rf']
    
    dfs_port = dfs_month[['month','permno','lmktcap','stk_month_ret','beta_group','rank_wgt','shrink_beta']]
    
    dfs_port['month'] = dfs_port['month'].astype(str)
    
    dfs_port = pd.merge(dfs_port, _ff[['month','rf']], on=['month'], how='inner')
    
    dfs_port.dropna(inplace=True)
    """
    Construct BAB portfolio
    
    Rescale both portfolios to have a beta of one at portfolio formation
    
    Self-financing zero-beta portfolio 
        long the low-beta portfolio and shortsell the high-beta portfolio
    """
    
    Beta = dfs_port.groupby(['month','beta_group']).apply(vector_vwt, 'shrink_beta','rank_wgt' ).reset_index(name='beta')
    Beta['lag_beta'] = Beta.groupby('beta_group')['beta'].shift(1)
    
    Beta_port = Beta.pivot(index='month', columns='beta_group', values='lag_beta').reset_index()
    Beta_port.rename(columns={'high': 'high_beta', 'low': 'low_beta'}, inplace=True)
    
    Return_port = dfs_port.groupby(['month','beta_group']).apply(vector_vwt, 'stk_month_ret','rank_wgt' ).reset_index(name='port_ret')
    Return_port = Return_port.pivot(index='month', columns='beta_group', values='port_ret').reset_index()
    Return_port.rename(columns={'high': 'high_ret', 'low': 'low_ret'}, inplace=True)
    
    Return_port.dropna(inplace=True)
    Beta_port.dropna(inplace=True)
    Return_port['month'] = Return_port['month'].astype(str)
    Beta_port['month'] = Beta_port['month'].astype(str)
    
    
    bab_port = pd.merge(Return_port, Beta_port, on=['month'], how='inner')
    
    bab_port = pd.merge(bab_port, _ff[['month','rf','mkt-rf']], on=['month'], how='left')
    
    bab_port['high_bab'] = 1 / bab_port['high_beta'] * (bab_port['high_ret'] - bab_port['rf'])
    bab_port['low_bab'] = 1 / bab_port['low_beta'] * (bab_port['low_ret'] - bab_port['rf'])
    
    bab_port['bab_ret'] = bab_port['low_bab'] - bab_port['high_bab']
    
    bab_port.dropna(inplace=True)
    bab_port.reset_index(inplace=True)
    bab_port = bab_port[['month','bab_ret']]

    Bab_Port.append(bab_port)

/var/folders/vt/x4z4r3fd0xg5nszdcjd244900000gn/T/ipykernel_1748/2811898436.py:32: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  mkt_ret = dfs_period.groupby('date').apply(wavg, 'ret', 'mkt_cap').reset_index(name='mkt_ret')
/var/folders/vt/x4z4r3fd0xg5nszdcjd244900000gn/T/ipykernel_1748/2811898436.py:122: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_month['lmktcap'] = dfs_month.groupby('permno')['mkt_cap'].shift(1)
/var/folders/vt/x4z4r3fd0xg5nszdcjd244

In [7]:
Bab_Port = pd.concat(Bab_Port, ignore_index=True)

Test = pd.merge(Bab_Port,BAB_AQR,how='left',on=['month'])
Test = Test.dropna(subset=['bab_ret','US_BAB'])

BAB_corr = stats.pearsonr(Test['bab_ret'], Test['US_BAB'])[0]
print(BAB_corr)

new_row = pd.DataFrame({'month': ['Correlation'], 'bab_ret': [BAB_corr]})
Bab_Port = pd.concat([Bab_Port, new_row], ignore_index=True)

Bab_Port.rename(columns={'bab_ret':'BAB Factor'}, inplace=True)

Bab_Port.to_excel('./BAB factor 1926-2013.xlsx', index=False, sheet_name='BAB factor from 1926-2013')


0.9595993350836289


In [8]:
"""
 A replication of Table 3 in Frazzini and Pedersen (2014)
"""

def Beta_group (row):
    if (row['beta0']<=row['shrink_beta']) and (row['shrink_beta']<row['beta10']):
        group='P1'
    elif (row['beta10']<=row['shrink_beta']) and (row['shrink_beta']<row['beta20']):
        group='P2'
    elif (row['beta20']<=row['shrink_beta']) and (row['shrink_beta']<row['beta30']):
        group='P3'
    elif (row['beta30']<=row['shrink_beta']) and (row['shrink_beta']<row['beta40']):
        group='P4'
    elif (row['beta40']<=row['shrink_beta']) and (row['shrink_beta']<row['beta50']):
        group='P5'
    elif (row['beta50']<=row['shrink_beta']) and (row['shrink_beta']<row['beta60']):
        group='P6'
    elif (row['beta60']<=row['shrink_beta']) and (row['shrink_beta']<row['beta70']):
        group='P7'
    elif (row['beta70']<=row['shrink_beta']) and (row['shrink_beta']<row['beta80']):
        group='P8'
    elif (row['beta80']<=row['shrink_beta']) and (row['shrink_beta']<row['beta90']):
        group='P9'
    elif (row['beta90']<=row['shrink_beta']) and (row['shrink_beta']<row['beta100']):
        group='P10'
    else:
        group=np.nan
    return group


dfs_ny = pd.concat(dfs_ny, ignore_index=True)
dfs_ny = dfs_ny.groupby(['month'])['shrink_beta'] \
    .describe(percentiles=[0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1]).reset_index()

dfs_ny = dfs_ny.rename(columns={'0%':'beta0', '10%':'beta10','20%':'beta20','30%':'beta30','40%':'beta40','50%':'beta50','60%':'beta60','70%':'beta70', '80%':'beta80', '90%':'beta90','100%':'beta100'})

dfs_ny = dfs_ny.drop(['count','mean','std','min','max'], axis=1)
dfs_decile = dfs_port[['month','permno','lmktcap','stk_month_ret','shrink_beta']]

dfs_decile['month'] = dfs_decile['month'].astype(str)
dfs_ny['month'] = dfs_ny['month'].astype(str)
dfs_decile = pd.merge(dfs_decile, dfs_ny, on=['month'], how='inner')

dfs_decile['beta_group'] = dfs_decile.apply(Beta_group, axis=1)

/var/folders/vt/x4z4r3fd0xg5nszdcjd244900000gn/T/ipykernel_1748/321046752.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs_decile['month'] = dfs_decile['month'].astype(str)


In [9]:
port_month_ret = dfs_decile.groupby(['beta_group','month']).apply(wavg,'stk_month_ret','lmktcap').reset_index(name='port_month_ret')

_ff['month'] = _ff['month'].astype(str)
port_month_ret['month'] = port_month_ret['month'].astype(str)

port_month_ret = pd.merge(port_month_ret, _ff[['month','rf']], on=['month'], how='inner')

port_month_ret['port_excess_ret'] = port_month_ret['port_month_ret']-port_month_ret['rf']

port_month_ret = port_month_ret.dropna(subset=['beta_group','port_excess_ret'], axis=0)

port = port_month_ret.pivot(index='month', columns='beta_group', values='port_excess_ret').reset_index()

bab_port['month'] = bab_port['month'].astype(str)
port = pd.merge(port, _ff, on='month', how='inner')
port = pd.merge(port, bab_port[['month','bab_ret']], on='month', how='inner')
port.dropna(inplace=True)

/var/folders/vt/x4z4r3fd0xg5nszdcjd244900000gn/T/ipykernel_1748/3405380354.py:1: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  port_month_ret = dfs_decile.groupby(['beta_group','month']).apply(wavg,'stk_month_ret','lmktcap').reset_index(name='port_month_ret')


In [10]:
def calculate_alpha(portfolio_return, factors):
    X = sm.add_constant(factors)
    model = sm.OLS(portfolio_return, X).fit()
    alpha = model.params['const']  # 截距项
    alpha_se = model.bse[0]
    alpha_t_stat = alpha / alpha_se
    return alpha, alpha_t_stat


Excess_returen = {}
Excess_tstats = {}
CAPM_alphas = {}
CAPM_tstats = {}

for column in ['P1', 'P2', 'P3', 'P4', 'P5', 'P6', 'P7', 'P8', 'P9', 'P10','bab_ret']:

    mean_excess_ret = port[column].mean()
    std_excess_ret = port[column].std()
    n = len(port[column])
    se_excess_ret = std_excess_ret / np.sqrt(n)
    t_stat_excess_ret = mean_excess_ret / se_excess_ret
    ret_tstat_with_brackets = f"({t_stat_excess_ret:.2f})"

    Excess_returen[column] = mean_excess_ret * 100
    Excess_tstats[column] = ret_tstat_with_brackets

    capm_alpha, capm_tstat = calculate_alpha(port[column], port[['mkt-rf']])
    capm_tstat_with_brackets = f"({capm_tstat:.2f})"
    CAPM_alphas[column] = capm_alpha * 100
    CAPM_tstats[column] = capm_tstat_with_brackets

# 将 alpha 和 t-statistics 转换为 DataFrame
excess_df = pd.DataFrame(Excess_returen, index=['Excess return'])
excess_tstat_df = pd.DataFrame(Excess_tstats, index=['     '])
capm_alpha_df = pd.DataFrame(CAPM_alphas, index=['CAPM alpha'])
capm_tstat_df = pd.DataFrame(CAPM_tstats, index=['     '])

# 合并成一张宽表
results_df = pd.concat([excess_df,excess_tstat_df,capm_alpha_df,capm_tstat_df],axis=0)
results_df.rename(columns={'bab_ret':'BAB', 'P1':'Low beta','P10':'High beta'}, inplace=True)

/var/folders/vt/x4z4r3fd0xg5nszdcjd244900000gn/T/ipykernel_1748/1589414078.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha_se = model.bse[0]
/var/folders/vt/x4z4r3fd0xg5nszdcjd244900000gn/T/ipykernel_1748/1589414078.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha_se = model.bse[0]
/var/folders/vt/x4z4r3fd0xg5nszdcjd244900000gn/T/ipykernel_1748/1589414078.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  alpha_s

In [11]:
results_df.to_excel('Table 3.xlsx', index=True, sheet_name='US equities returns, 1926-2024')